### Import Statements

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import imageio
from imageio import v3 as Im
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from keras import backend as K
import sys
import tensorflow as tf

### GPU Check

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')

In [3]:
gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
#K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

### Loading Excel to DataFrame

In [6]:
df = pd.read_excel('/Thesis/Datasets/SAMM_Sample.xlsx', index_col=0)

In [7]:
df.count()

Filename             159
Inducement Code      159
Onset Frame          159
Apex Frame           159
Offset Frame         159
Duration             159
Micro                159
Action Units         159
Estimated Emotion    159
Objective Classes    159
Notes                 61
dtype: int64

In [8]:
df.head()

,Filename,Inducement Code,Onset Frame,Apex Frame,Offset Frame,Duration,Micro,Action Units,Estimated Emotion,Objective Classes,Notes
Subject,,,,,,,,,,,
6,006_1_2,1,5562,5588,5632,71,Micro - 1/2,4+7,Anger,3,NaN
6,006_1_3,1,3912,3948,3988,77,Micro - 1/2,4,Anger,3,While blinking
6,006_1_4,1,2324,2368,2403,80,Micro - 1/2,4+7,Anger,3,NaN
6,006_1_5,1,5343,5388,5424,82,Micro - 1/2,4+7,Anger,3,NaN
6,006_1_6,1,7160,7197,7259,100,Micro - 1/2,4+7+43,Anger,7,Double AU4


In [8]:
positive = df[df['Estimated Emotion'] == 'Happiness']['Filename']
print('Positive :', positive.count())

negative = df[(df['Estimated Emotion'] == 'Anger') | (df['Estimated Emotion'] == 'Sadness') | (df['Estimated Emotion'] == 'Fear') | (df['Estimated Emotion'] == 'Disgust') | (df['Estimated Emotion'] == 'Contempt')]['Filename']
print('Negative :',negative.count())

surprise = df[df['Estimated Emotion'] == 'Surprise']['Filename']
print('Surprise :',surprise.count())

Positive : 26
Negative : 92
Surprise : 15


### Initialization

In [9]:
subjectPath = '/Thesis/Datasets/SAMM'

In [10]:
image_rows, image_columns, frames_Count = 64, 64, 40
training_samm_list = []

### SAMM Dataset

In [11]:
for lab in [positive, negative, surprise]:
    subDirectory = lab#.tolist() 
    count = 0
    for sub in subDirectory:
        labelpath = '/Thesis/Datasets/SAMM/'+ sub[:3] +'/'# + sub + '/'
        directorylisting = os.listdir(labelpath)
        #count = count + 1
        #print(directorylisting)
        #break
        for video in directorylisting:
            videopath = labelpath + video
            frames = []
            framelisting = os.listdir(videopath)
            #loadedvideo = Im.get_reader(videopath, format='FFMPEG' )#, 'ffmpeg')
            if len(framelisting) > frames_Count:
                framerange = [x  for x in range(frames_Count)]
                count = count + 1
                for frame in framerange:
                    #image = loadedvideo.get_data(frame)
                    imagepath = videopath + "/" + framelisting[frame]
                    image = cv2.imread(imagepath)
                    imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
                    grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                    frames.append(grayimage)
                    # break
                frames = np.asarray(frames)
                videoarray = np.rollaxis(np.rollaxis(frames, 2, 0), 2, 0)
                training_samm_list.append(videoarray)
                # break
    
    print(count)
    # break

290
744
96


In [12]:
len(training_samm_list)

1130

In [13]:
290+744

1034

### Creating Target labels

In [15]:
training_samm_list = np.asarray(training_samm_list)
training_samm_samples = len(training_samm_list)

training_samm_labels = np.zeros((training_samm_samples, ), dtype = int)

training_samm_labels[0:290] = 0
training_samm_labels[290:1034] = 1
training_samm_labels[1034:1130] = 2

training_samm_labels = np_utils.to_categorical(training_samm_labels, 3)

In [17]:
training_samm_data = [training_samm_list, training_samm_labels]
(training_frames_samm, traininglabels_samm) = (training_samm_data[0], training_samm_data[1])
training_samm_set = np.zeros((training_samm_samples, 1, image_rows, image_columns, frames_Count))
for h in range(training_samm_samples):
    training_samm_set[h][0][:][:][:] = training_frames_samm[h,:,:,:]

In [19]:
training_samm_set = training_samm_set.astype('float32')
training_samm_set -= np.mean(training_samm_set)
training_samm_set /= np.max(training_samm_set)

In [20]:
training_samm_set.shape

(1130, 1, 64, 64, 40)

#### Save Numpy Arrays

In [21]:
# Save Nump Arrays to save time
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexp_samm_images.npy', training_samm_set)
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexp_samm_labels.npy', training_samm_labels)

#### Loading Numpy arrays

In [22]:
training_samm_set = None
training_samm_labels = None
training_samm_set  = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexp_samm_images.npy')
training_samm_labels = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexp_samm_labels.npy')

training_samm_set.shape

(1130, 1, 64, 64, 40)

### Model

In [23]:
model = Sequential()
model.add(Convolution3D(32, (3, 3, 15), input_shape=(1, image_rows, image_columns, frames_Count), activation='relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 32, 62, 62, 26)    4352      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 32, 20, 20, 8)     0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 20, 20, 8)     0         
_________________________________________________________________
flatten (Flatten)            (None, 102400)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               13107328  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3

In [12]:
# Load pre-trained weights

#model.load_weights('/Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn/weights-improvement-53-0.88.hdf5')

#model.load_weights('/Thesis/MicroExpSTCNN/CASME_SQUARE/weights-improvement-53-0.88.hdf5')

#### Creating checkpoints

In [24]:
filepath="/Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#### Spliting the dataset into training and validation sets

In [25]:
train_images, validation_images, train_labels, validation_labels =  train_test_split(training_samm_set, training_samm_labels, test_size=0.2, random_state=4)

#### Save validation set in a numpy array

In [26]:
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_images.npy', validation_images)
np.save('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_labels.npy', validation_labels)

#### Load validation set from numpy array

In [27]:
validation_images = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_images.npy')
validation_labels = np.load('/Thesis/MicroExpSTCNN/Training_dataset/microexpstcnn_val_labels.npy')

#### Pre-Trained data

In [15]:
#validation_images = np.load('/Thesis/MicroExpSTCNN/CASME_SQUARE/microexpstcnn_val_images.npy')
#validation_labels = np.load('/Thesis/MicroExpSTCNN/CASME_SQUARE/microexpstcnn_val_labels.npy')

#### Training the model

In [28]:
hist = model.fit(train_images, train_labels, validation_data = (validation_images, validation_labels), callbacks=callbacks_list, batch_size = 16, epochs = 100, shuffle=True)


Epoch 1/100
57/57 [==============================] - 6s 42ms/step - loss: 0.7970 - accuracy: 0.6571 - val_loss: 0.6938 - val_accuracy: 0.7257

Epoch 00001: val_accuracy improved from -inf to 0.72566, saving model to /Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn\weights-improvement-01-0.73.hdf5
Epoch 2/100
57/57 [==============================] - 2s 36ms/step - loss: 0.7072 - accuracy: 0.7035 - val_loss: 0.6083 - val_accuracy: 0.7345

Epoch 00002: val_accuracy improved from 0.72566 to 0.73451, saving model to /Thesis/MicroExpSTCNN/Training_dataset/weights_microexpstcnn\weights-improvement-02-0.73.hdf5
Epoch 3/100
57/57 [==============================] - 2s 36ms/step - loss: 0.6520 - accuracy: 0.7046 - val_loss: 0.6018 - val_accuracy: 0.6549

Epoch 00003: val_accuracy did not improve from 0.73451
Epoch 4/100
57/57 [==============================] - 2s 36ms/step - loss: 0.6389 - accuracy: 0.7124 - val_loss: 0.6344 - val_accuracy: 0.7345

Epoch 00004: val_accuracy did not im

#### Finding Confusion Matrix

In [30]:
predictions = model.predict(validation_images)
predictions_labels = np.argmax(predictions, axis=1)
validation_labels = np.argmax(validation_labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)

[[ 23  35   0]
 [  9 142   0]
 [  5  10   2]]
